# Sep 2, 2024: visualize soft-communities: nifti + 3d renderings

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random

import arviz as az

import ants
from nipype.interfaces import afni
from brainglobe_utils.IO.image.load import load_nii

from brainrender import Scene, actor
import vedo
vedo.settings.default_backend = 'k3d'
from vedo import Volume, Plotter

from itertools import product, combinations
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

PARC_DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson-corr'
args.THRESHOLDING = f'positive'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 10
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'ses'

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = f'{BASE_path}/roi_results_v2/{PARC_DESC}'
TS_path = f'{ROI_path}/runwise_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLDING}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [5]:
parcels_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')
parcels = parcels_img.numpy()
roi_labels = np.loadtxt(f'{PARCELS_path}/{PARC_DESC}_desc-labels.txt')

In [6]:
parcels_img


ANTsImage (LPI)
         Pixel Type : float (float32)
         Components : 1
         Dimensions : (58, 79, 45)
         Spacing    : (0.2, 0.2, 0.2)
         Origin     : (18.1, 2.7, -7.8)
         Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [7]:
def concatenate(in_files, out_file):
    try:
        os.remove(out_file)
    except:
        pass

    tcat = afni.TCat()
    tcat.inputs.in_files = in_files
    tcat.inputs.out_file = out_file
    tcat.inputs.rlt = ''
    tcat.cmdline 
    tcat.run()

    for file in in_files:
        try:
            os.remove(file)
        except:
            pass
    return None

def soft_comms_to_nifti(args, X, folder, level=-1):
    args.num_rois, args.num_comms = X.shape

    in_files = []
    for idx_comm in range(args.num_comms):
        x = X[:, idx_comm]
        x_img = np.zeros_like(parcels)
        for idx, roi in enumerate(roi_labels):
            x_img += (parcels == roi) * (x[idx])
        
        file = f'{folder}/comm-{idx_comm}_desc-3d.nii.gz'
        parcels_img.new_image_like(x_img).to_filename(file)
        in_files.append(file)

    if level == -1:
        out_file = f'{folder}/desc-3d.nii.gz'
    else:
        out_file = f'{folder}/level-{level}_desc-3d.nii.gz'
    concatenate(in_files, out_file)
    return None

def nested_soft_comms_to_nifti(args, soft_comms, folder):
    for level in range(len(soft_comms)):
        soft_comms_to_nifti(args, soft_comms[level], folder, level=level)

In [8]:
sbm = (False, 'd')
args.dc, args.sbm = sbm

args.nested = args.sbm == 'h'

args.force_niter = 40000
args.num_draws = int((1/2) * args.force_niter)

def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['a'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
print(SBM)

sbm-nd-d


In [9]:
folders = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/soft-communities/{SBM}', recursive=True))
print(folders)

['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-10/layer-individual/unit-ses/estimates/individual/sub-SLC01/soft-communities/sbm-nd-d', '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-10/layer-individual/unit-ses/estimates/individual/sub-SLC02/soft-communities/sbm-nd-d', '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-10/layer-individual/unit-ses/estimates/individual/sub-SLC03/soft-communities/sbm-nd-d', '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-10/layer-individual/unit-se

In [10]:
for folder in tqdm(folders):
    with open(f'{folder}/desc-matrices.pkl', 'rb') as f:
        if args.sbm in ['a', 'd']:
            [Grps_mat, soft_comms] = pickle.load(f)
            soft_comms_to_nifti(args, soft_comms, folder)
        elif args.sbm in ['h']:
            [Grps_mats, soft_comms] = pickle.load(f)
            nested_soft_comms_to_nifti(args, soft_comms, folder)

  0%|          | 0/10 [00:00<?, ?it/s]

240907-11:22:11,859 nipype.interface INFO:
	 stderr 2024-09-07T11:22:11.858968:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240907-11:22:11,864 nipype.interface INFO:
	 stderr 2024-09-07T11:22:11.864570:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240907-11:22:11,865 nipype.interface INFO:
	 stderr 2024-09-07T11:22:11.865757:*+ WARNING: Set TR of output dataset to 1.0 s
240907-11:22:11,968 nipype.interface INFO:
	 stderr 2024-09-07T11:22:11.968267:++ elapsed time = 0.1 s


 10%|█         | 1/10 [00:01<00:13,  1.49s/it]

240907-11:22:13,335 nipype.interface INFO:
	 stderr 2024-09-07T11:22:13.335541:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240907-11:22:13,346 nipype.interface INFO:
	 stderr 2024-09-07T11:22:13.346886:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240907-11:22:13,348 nipype.interface INFO:
	 stderr 2024-09-07T11:22:13.348805:*+ WARNING: Set TR of output dataset to 1.0 s
240907-11:22:13,478 nipype.interface INFO:
	 stderr 2024-09-07T11:22:13.478229:++ elapsed time = 0.1 s


 20%|██        | 2/10 [00:03<00:12,  1.51s/it]

...

 80%|████████  | 8/10 [00:12<00:03,  1.51s/it]

240907-11:22:23,986 nipype.interface INFO:
	 stderr 2024-09-07T11:22:23.985972:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240907-11:22:23,990 nipype.interface INFO:
	 stderr 2024-09-07T11:22:23.990162:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240907-11:22:23,991 nipype.interface INFO:
	 stderr 2024-09-07T11:22:23.991382:*+ WARNING: Set TR of output dataset to 1.0 s
240907-11:22:24,100 nipype.interface INFO:
	 stderr 2024-09-07T11:22:24.100234:++ elapsed time = 0.1 s


 90%|█████████ | 9/10 [00:13<00:01,  1.51s/it]

240907-11:22:25,467 nipype.interface INFO:
	 stderr 2024-09-07T11:22:25.467039:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240907-11:22:25,474 nipype.interface INFO:
	 stderr 2024-09-07T11:22:25.474694:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240907-11:22:25,476 nipype.interface INFO:
	 stderr 2024-09-07T11:22:25.476290:*+ WARNING: Set TR of output dataset to 1.0 s
240907-11:22:25,587 nipype.interface INFO:
	 stderr 2024-09-07T11:22:25.586941:++ elapsed time = 0.1 s


100%|██████████| 10/10 [00:15<00:00,  1.51s/it]


In [11]:
def get_colors(args, X):
    rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))
    cmap = cc.cm.CET_L12 #mpl.colormaps['Blues'] #mpl.colormaps['viridis']
    vals_list = np.unique(X)
    colors_list = cmap(rescale(vals_list))
    colors_list = [colors.to_hex(c) for c in colors_list]
    colors_df = pd.DataFrame({
        'val':vals_list,
        'color':colors_list,
    })
    return colors_df

def add_vol(args, scene, vol, color='#cccccc', alpha=0.2, label='template'):
    mesh = vol.isosurface()
    mesh = mesh.smooth(
        niter=1000, 
        pass_band=0.1, 
        edge_angle=15, 
        feature_angle=150,
        boundary=False,
    )
    mesh_actor = actor.Actor(
        mesh, 
        name=f'{label}', 
        br_class='Volume', 
        color=color, 
        alpha=alpha,
    )
    scene.add(mesh_actor)
    # scene.add_label(actor=mesh_actor, label=label)
    return scene

def visualize_soft_comm(args, template, data_vol, colors_df, folder, idx_comm, level=-1):
        # scene
        scene = Scene()
        root_brain = scene.get_actors()[0].alpha(0.0)
        
        spacing=[args.spacing]*3
        
        # brain template
        vol = Volume(template, origin=[-75,0,0], spacing=spacing)
        scene = add_vol(
            args, scene, vol=vol, 
            color='#cccccc', alpha=0.2,
            label='template'
        )
        
        # brain data for a community
        if level == -1:
            os.system(f'mkdir -p {folder}/brainrender')
            filename = f'{folder}/brainrender/comm-{idx_comm:02d}.html'
        else:
            os.system(f'mkdir -p {folder}/brainrender/level-{level}')
            filename = f'{folder}/brainrender/level-{level}/comm-{idx_comm:02d}.html'
        
        # for val in np.unique(data_vol)[1:]:
        for roi in roi_labels:
            val = np.unique((parcels == roi) * data_vol)
            val = val[1] if len(val) > 1 else val
            alpha = 0.05 if val == 0.0 else 1.0
            clr = colors_df.iloc[(colors_df['val']-val).abs().argsort()[0]]['color']
            clr = '#cccccc' if val == 0.0 else clr
            roi_vol = (parcels == roi)
            roi_vol = Volume(roi_vol, origin=[0,0,0], spacing=spacing)
            scene = add_vol(
                args, scene, vol=roi_vol, 
                color=clr,
                alpha=alpha,
                label=f'roi {roi}',
            )
        scene.export(f'{filename}')
        return None

def visualize_soft_comms(args, template, data, colors_df, folder, level=-1):
    if len(data.shape) > 3:
        for idx_comm in range(data.shape[-1]):
            visualize_soft_comm(args, template, data[:, :, :, idx_comm], colors_df, folder, idx_comm, level)
    else:
        visualize_soft_comm(args, template, data, colors_df, folder, 0, level)
    return None

In [12]:
def combine_soft_htmls(args, html_folder):
    # Create the main HTML content
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>HTML Grid with Headings</title>
        <style>
            body {
                display: grid;
                grid-template-columns: repeat(auto-fill, minmax(1000px, 1fr));
                gap: 20px;
                padding: 20px;
            }
            iframe {
                width: 850px;
                height: 850px;
                border: none;
            }
            .grid-item {
                text-align: center;
                margin-bottom: 10px;
            }
            h2 {
                margin: 0;
            }
        </style>
    </head>
    <body>
    """
    # Add iframes and headings for each HTML file
    html_files = sorted(glob.glob(f'{html_folder}/comm*', recursive=True))
    for html_file in html_files:
        fs = html_file.split('/')
        # print(fs)
        if args.sbm in ['a', 'd']:
            parts = [-5, -3, -1]
        if args.sbm in ['h']:
            parts = [-6, -4, -2, -1]
        # Extract filename (without extension) for heading 
        heading = f'<h2>{"_".join([fs[i] for i in parts])}</h2>'  # Capitalize the filename for title
        src = html_file.split('/')[-1]
        # src = Path(html_file).resolve().as_uri()
        iframe_tag = f'<div class="grid-item">{heading}<iframe src="{src}"></iframe></div>'
        html_content += iframe_tag
    # Close the HTML content
    html_content += """
    </body>
    </html>
    """
    with open(f'{html_folder}/index.html', 'w') as f:
        f.write(html_content)
    print("index.html has been created successfully.")

In [13]:
for folder in tqdm(folders[:]):
    args.spacing = 400 #um # 200
    template = f'{BASE_path}/gabe_symmetric_N162/Symmetric_N162_0.20_RAS.nii.gz'
    template = load_nii(template, as_array=True)

    if args.sbm in ['a', 'd']:
        file = f'{folder}/desc-3d.nii.gz'
        data = load_nii(file, as_array=True)
        colors_df = get_colors(args, data)
        visualize_soft_comms(args, template, data, colors_df, folder)

    if args.sbm in ['h']:
        files = sorted(glob.glob(f'{folder}/*.nii.gz', recursive=True))
        for level, file in enumerate(files):
            # file = f'{folder}/level-{level}_desc-3d.nii.gz'
            data = load_nii(file, as_array=True)
            colors_df = get_colors(args, data)
            visualize_soft_comms(args, template, data, colors_df, folder, level)

  0%|          | 0/10 [00:00<?, ?it/s]

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

The brainrender scene has been exported for web. The results are saved at 
/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructe
d/method-pearson-corr/threshold-positive/edge-binary/density-10/layer-individual/unit-ses/estimates/individual/sub-
SLC01/soft-communities/sbm-nd-d/brainrender/comm-00.html

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

...

The brainrender scene has been exported for web. The results are saved at 
/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructe
d/method-pearson-corr/threshold-positive/edge-binary/density-10/layer-individual/unit-ses/estimates/individual/sub-
SLC10/soft-communities/sbm-nd-d/brainrender/comm-15.html

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

The brainrender scene has been exported for web. The results are saved at 
/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructe
d/method-pearson-corr/threshold-positive/edge-binary/density-10/layer-individual/unit-ses/estimates/individual/sub-
SLC10/soft-communities/sbm-nd-d/brainrender/comm-16.html

100%|██████████| 10/10 [37:12<00:00, 223.21s/it]


In [14]:
files = sorted(glob.glob(f'{folder}/*.nii.gz', recursive=True))
print(files)

['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-10/layer-individual/unit-ses/estimates/individual/sub-SLC10/soft-communities/sbm-nd-d/desc-3d.nii.gz']


In [15]:
for folder in tqdm(folders[:]):
    if args.sbm in ['a', 'd']:
        html_folders = sorted(glob.glob(f'{folder}/brainrender', recursive=True))
    if args.sbm in ['h']:
        html_folders = sorted(glob.glob(f'{folder}/brainrender/level*', recursive=True))

    for html_folder in html_folders:
        combine_soft_htmls(args, html_folder)
        # break

100%|██████████| 10/10 [00:00<00:00, 1775.07it/s]

index.html has been created successfully.
index.html has been created successfully.
index.html has been created successfully.
index.html has been created successfully.
index.html has been created successfully.
index.html has been created successfully.
index.html has been created successfully.
index.html has been created successfully.
index.html has been created successfully.
index.html has been created successfully.
